In [1]:
#Импорт библиотек
import pandas as pd
import gc
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import os
from joblib import dump, load

#Список имён файлов под features------------------
files = ["wagons_probeg_ownersip.parquet", "dislok_wagons.parquet",
         "wag_params.parquet", "pr_rems.parquet", 
         "tr_rems.parquet", "kti_izm.parquet",
         "freight_info.parquet", "stations.parquet"]

folders = ['train', 'test']
relative_paths = [os.path.join(folder, file) for folder in folders for file in files]

In [2]:
def processing_data(folder_name, files):
    relative_paths = [os.path.join(folder_name, file) for file in files]
    df_wagons_probeg_ownership, df_dislok_wagons, df_wag_params, df_pr_rems, df_tr_rems, df_kti_izm, df_freight_info, df_stations = list(map(pd.read_parquet, relative_paths))

    #Не использую
    df_freight_info = None
    df_stations = None

    #Избавляемся от лишних признаков в probeg
    df1 = df_wagons_probeg_ownership.sort_values(by=['repdate', 'wagnum'], ascending=True)
    df1 = df1.drop(columns = ['rod_id', 'month','ownership_type', 
                            'manage_type'])
    probeg = df1
    #print(probeg.head(5))
    print(len(list(set(probeg['wagnum'].values))), probeg.shape) #Проверка на количество информации про вагоны 33977/33977 вагонов

    #Избавляемся от лишних признаков в dislok
    df2 = df_dislok_wagons.sort_values(by=['plan_date', 'wagnum'], ascending=True)
    df2 = df2.drop(columns = ['st_id_send', 'id_road_send', 'isload', 'fr_id', 
                        'last_fr_id', 'distance', 'id_road_disl', 'kod_vrab',
                        'date_pl_rem', 'st_id_dest', 'id_road_dest', 'ost_prob'])
    dislok = df2
    #print(dislok.head(5))
    print(len(list(set(dislok['wagnum'].values))), dislok.shape) #Проверка на количество информации про вагоны 33977/33977 вагонов

    df3 = df_wag_params.sort_values(by=['wagnum'], ascending=True)
    df3 = df3.drop(columns = ['cnsi_volumek', 'ownertype', 'zavod_build', 'model', 'rod_id', 'date_iskl'])
    df3 = df3.drop_duplicates(subset='wagnum', keep='last')
    descr_wagon = df3
    #print(descr_wagon.head(3))
    print(len(list(set(descr_wagon['wagnum'].values))), descr_wagon.shape) #Проверка на количество информации про вагоны 33977/33977 вагонов

    df4 = df_pr_rems.sort_values(by=['wagnum'], ascending=True)
    df4 = df4.drop(columns = ['road_id_send', 'road_id_rem', 'st_id_send', 'st_id_rem', 
            'month', 'kod_vrab', 'rod_id', 'model'])
    plans_rem = df4
    #print(plans_rem.head(10))
    print(len(list(set(plans_rem['wagnum'].values))), plans_rem.shape) #Проверка на количество информации про вагоны 10393/33977 вагонов
    #Использовать этот df мы не будем так как содержит информацию не про все вагоны

    df5 = df_tr_rems.sort_values(by=['wagnum'], ascending=True)
    df5 = df5.drop(columns = ['kod_vrab', 'st_id_send', 'por_probeg', 'road_id_send'])
    haved_rem = df5
    #print(haved_rem.head(10))
    print(len(list(set(haved_rem['wagnum'].values))), haved_rem.shape) #Проверка на количество информации про вагоны 17849/33977 вагонов
    #Использовать этот df мы не будем так как содержит информацию не про все вагоны

    df6 = df_kti_izm.sort_values(by=['operation_date_dttm', 'wagnum'], ascending=True)
    df6 = df6.drop(columns = [])
    kti = df6
    #print(kti.head())
    print(len(list(set(kti['wagnum'].values))), kti.shape) #Проверка на количество информации про вагоны 6829/33977 вагонов
    #Использовать этот df мы не будем так как содержит информацию не про все вагоны

    #Будем использовать информацию только ту, в которой описан каждый вагон
    data = probeg.merge(dislok, left_on=['repdate', 'wagnum'], right_on=['plan_date', 'wagnum'])\
                .drop(columns=['plan_date'])\
                .merge(descr_wagon, left_on='wagnum', right_on='wagnum')\
                .sort_values(by=['wagnum', 'repdate'], ascending=True)
    
    print(len(list(set(data['wagnum'].values))), data.shape)
    data.to_csv("data_" + folder_name + ".csv")
    return data

In [3]:
Data_train = processing_data('train', files)
Data_test = processing_data('test', files)

33977 (6249857, 4)
33977 (6250933, 4)
33977 (33977, 14)
10393 (10441, 3)
17849 (48652, 13)
6829 (67967, 19)
33977 (6249316, 19)
33708 (943810, 4)
33701 (977304, 4)
33977 (33977, 14)
1589 (1591, 3)
3922 (7500, 13)
6686 (22428, 19)
33701 (943591, 19)


In [4]:
def load_Y(data, y_folder):
    print('Загрузка данных...')
    df_target = pd.read_csv(y_folder)
    df_features = data

    # Объединение данных
    df_target['month'] = pd.to_datetime(df_target['month'])
    df_features['repdate'] = pd.to_datetime(df_features['repdate'])
    df_combined = pd.merge(df_target, df_features, left_on=['wagnum', 'month'], right_on=['wagnum', 'repdate'])
    
    print("Объём Y target:", df_target.shape)
    print("Объём features:", df_features.shape)
    print("Объём пересечения features и Y target", df_combined.shape)
    print(f'Модель сможет обработать: {df_combined.shape[0]/df_target.shape[0]}% от Y\n',
        f'Количество необработанных объектов: { df_target.shape[0] - df_combined.shape[0]}')
    date_columns = ['repdate', 'month', 'date_kap', 'date_dep', 'date_build', 'srok_sl']

    # Преобразование к типу datetime, если необходимо
    df_combined[date_columns] = df_combined[date_columns].apply(pd.to_datetime, errors='coerce')

    # Преобразование в секунды
    for col in date_columns:
        df_combined[col] = (df_combined[col] - pd.Timestamp("1970-01-01")).dt.total_seconds()

    print('Подготовка данных...')

    X = df_combined.drop(['target_month', 'target_day'], axis=1)
    X = X.apply(lambda x: x.fillna(x.value_counts().index[0]))
    y_10_days = df_combined['target_day']
    y_month = df_combined['target_month']

    #print(df_combined.head()
    print('Выполнено!')
    return X, y_10_days, y_month

In [5]:
X_train, y_10_days_train, y_month_train = load_Y(Data_train, "train\\y_train.csv")
X_test, y_10_days_test, y_month_test = load_Y(Data_test , "test\\y_test.csv")
X_train, X_valid, y_10_days_train, y_10_days_valid, y_month_train, y_month_valid = train_test_split(X_train, y_10_days_train, y_month_train, test_size=0.3, random_state=42)

Загрузка данных...
Объём Y target: (203853, 4)
Объём features: (6249316, 19)
Объём пересечения features и Y target (203844, 22)
Модель сможет обработать: 0.9999558505393592% от Y
 Количество необработанных объектов: 9
Подготовка данных...
Выполнено!
Загрузка данных...
Объём Y target: (33708, 4)
Объём features: (943591, 19)
Объём пересечения features и Y target (33701, 22)
Модель сможет обработать: 0.9997923341639967% от Y
 Количество необработанных объектов: 7
Подготовка данных...
Выполнено!


In [6]:
model_10_days = RandomForestClassifier()
model_month = RandomForestClassifier()

In [7]:
def model_fit(model, X, Y):
    model.fit(X, Y)
     
def model_check(model, X, y_true):
    y_pred = model.predict(X)
    print("Accuracy:", accuracy_score(y_true, y_pred))
    return y_pred

In [8]:
print('Обучение моделей...')
model_fit(model_10_days, X_train, y_10_days_train)
model_fit(model_month, X_train, y_month_train)

print('Валидация моделей...')
model_check(model_10_days, X_valid, y_10_days_valid)
model_check(model_month, X_valid, y_month_valid)

dump(model_month, 'modelM.joblib')
dump(model_10_days, 'modelD.joblib')
#Загрузка обученных на тренировке моделей
modelM = load('modelM.joblib')
modelD = load('modelD.joblib')

Обучение моделей...
Валидация моделей...
Accuracy: 0.9836968963600091
Accuracy: 0.9575988488079276


In [9]:
print('Тестирование моделей...')
predictions_10_days = model_check(model_10_days, X_test, y_10_days_test)
predictions_month = model_check(model_month, X_test, y_month_test)

Тестирование моделей...
Accuracy: 0.9837393549152844
Accuracy: 0.9564107889973591


In [10]:
model_fit(model_10_days, X_test, y_10_days_test)
model_fit(model_month, X_test, y_month_test)

In [11]:
#Сохранение предсказаний
df_X_selected = X_test[['wagnum', 'month']]

df_X_selected.loc[:, 'month'] = pd.to_datetime(df_X_selected['month'], unit='s')# + pd.Timedelta("1970-01-01")

df_10_days_selected = pd.DataFrame(predictions_10_days, columns=['target_day'])[['target_day']]

df_month_selected = pd.DataFrame(predictions_month, columns=['target_month'])[['target_month']]

df_Y = pd.concat([df_X_selected, df_month_selected, df_10_days_selected], axis=1)

#print(df_Y)
df_Y.to_csv("Y_test_predict.csv", index=False)
#return calc_f1_score